In [1]:
# The Slimcoin Blockchain Simulation
# This script combines the blockchain and miner concepts to create a single,
# real-time simulation of a cryptocurrency network.

# It uses two main components:
# 1. A Blockchain class that manages the ledger and transactions.
# 2. A Miner class that runs in a separate thread to find new blocks.

# This is a conceptual tool for educational purposes only.

import hashlib
import json
import time
import random
import threading
import sys
import queue

# The target difficulty for the mining puzzle.
# A higher number of leading zeros means more difficult mining.
DIFFICULTY = 4

class Block:
    """Represents a single block in the Slimcoin blockchain."""
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        """Calculates the cryptographic hash of the block's content."""
        block_string = json.dumps({
            "index": self.index,
            "transactions": self.transactions,
            "timestamp": self.timestamp,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def __str__(self):
        """Provides a user-friendly string representation of the Block."""
        return (f"Block #{self.index}\n"
                f"Previous Hash: {self.previous_hash}\n"
                f"Timestamp: {time.ctime(self.timestamp)}\n"
                f"Transactions: {json.dumps(self.transactions, indent=2)}\n"
                f"Nonce: {self.nonce}\n"
                f"Hash: {self.hash}\n")

class Blockchain:
    """Manages the entire Slimcoin ledger."""
    def __init__(self):
        self.chain = []
        self.pending_transactions = []
        # Use a queue to safely pass newly mined blocks from the miner thread
        self.mined_block_queue = queue.Queue()
        self.create_genesis_block()

    def create_genesis_block(self):
        """Creates the very first block in the blockchain."""
        genesis_block = Block(0, ["Genesis Block"], time.time(), "0")
        self.chain.append(genesis_block)
        print("Genesis block created for Slimcoin.")

    def get_last_block(self):
        """Returns the most recent block in the chain."""
        return self.chain[-1]

    def add_transaction(self, transaction):
        """Adds a new transaction to the list of pending transactions."""
        self.pending_transactions.append(transaction)

    def add_mined_block(self, block_data):
        """
        Adds a new block to the chain after it has been mined.
        This is called by the main network loop.
        """
        # We need to make sure the block is valid before adding it to the chain
        if not self.is_valid_block(block_data):
            print("❌ Error: Invalid block received from miner.")
            return

        new_block = Block(
            index=block_data['index'],
            transactions=block_data['transactions'],
            timestamp=block_data['timestamp'],
            previous_hash=block_data['previous_hash']
        )
        new_block.nonce = block_data['nonce']
        new_block.hash = new_block.calculate_hash()

        print(f"\n✅ Block #{new_block.index} successfully added to the chain!")
        print(new_block)
        print("---------------------------------------------------\n")

        self.chain.append(new_block)
        self.pending_transactions = []

    def is_valid_block(self, block_data):
        """
        Checks if the mined block is valid.
        This is a simplified validation for demonstration.
        """
        # Check if the previous hash matches the last block's hash.
        if block_data['previous_hash'] != self.get_last_block().hash:
            return False

        # Check if the hash meets the difficulty target.
        if not block_data['hash'].startswith("0" * DIFFICULTY):
            return False

        return True

class Miner(threading.Thread):
    """A miner that performs Proof-of-Work in a separate thread."""
    def __init__(self, mining_job_queue, mined_block_queue):
        """
        Initializes the miner thread with queues for communication.
        `mining_job_queue`: receives jobs from the main blockchain loop.
        `mined_block_queue`: sends completed blocks back to the main loop.
        """
        super().__init__()
        self.mining_job_queue = mining_job_queue
        self.mined_block_queue = mined_block_queue
        self.running = True

    def run(self):
        """The main loop for the miner thread."""
        print("⛏️ Miner started. Waiting for new jobs...")
        while self.running:
            try:
                # Get a new mining job. This will block until a job is available.
                job = self.mining_job_queue.get(timeout=1)
                self.process_job(job)
            except queue.Empty:
                # If the queue is empty, the miner just waits for a new job.
                continue

    def process_job(self, job):
        """Performs the Proof-of-Work for a given job."""
        nonce = 0
        while self.running:
            block_string = json.dumps({
                "index": job.index,
                "transactions": job.transactions,
                "timestamp": time.time(),
                "previous_hash": job.previous_hash,
                "nonce": nonce
            }, sort_keys=True).encode()

            current_hash = hashlib.sha256(block_string).hexdigest()

            # Check if the hash meets the difficulty target.
            if current_hash.startswith("0" * DIFFICULTY):
                # We found a solution!
                mined_data = {
                    'index': job.index,
                    'transactions': job.transactions,
                    'timestamp': time.time(),
                    'previous_hash': job.previous_hash,
                    'nonce': nonce,
                    'hash': current_hash
                }
                # Put the completed block in the queue for the main thread
                self.mined_block_queue.put(mined_data)
                self.mining_job_queue.task_done()
                return # Exit the mining loop for this job

            nonce += 1

            # Provides live feedback on the mining process in the terminal.
            sys.stdout.write(f"\r⛏️ Mining... nonce: {nonce:,}, hash: {current_hash[:20]}...")
            sys.stdout.flush()

# --- Main Slimcoin Network Loop ---
def main():
    """The central loop that runs the Slimcoin network simulation."""
    my_slimcoin = Blockchain()

    # Queues for communication between the main thread and the miner thread.
    mining_job_queue = queue.Queue()
    mined_block_queue = queue.Queue()

    # Start the miner thread.
    miner_thread = Miner(mining_job_queue, mined_block_queue)
    miner_thread.daemon = True
    miner_thread.start()

    user_pool = ["Alice", "Bob", "Charlie", "David"]

    try:
        while True:
            # Check for newly mined blocks from the miner thread
            if not mined_block_queue.empty():
                mined_block_data = mined_block_queue.get()
                my_slimcoin.add_mined_block(mined_block_data)
                mined_block_queue.task_done()

            # Simulate new transactions arriving at random intervals
            time.sleep(random.uniform(0.5, 2))

            sender = random.choice(user_pool)
            receiver = random.choice(user_pool)
            if sender == receiver:
                continue

            amount = random.randint(1, 20)
            new_transaction = {
                "from": sender,
                "to": receiver,
                "amount": amount
            }
            my_slimcoin.add_transaction(new_transaction)
            print(f"💰 New transaction: {new_transaction}")

            # When there are enough transactions, create a mining job.
            if len(my_slimcoin.pending_transactions) >= 3:
                last_block = my_slimcoin.get_last_block()
                mining_job = {
                    "index": last_block.index + 1,
                    "transactions": my_slimcoin.pending_transactions[:],
                    "previous_hash": last_block.hash
                }
                # Put the job in the queue for the miner to pick up.
                mining_job_queue.put(mining_job)

    except KeyboardInterrupt:
        print("\n\nSlimcoin network simulation stopped by user. Goodbye!")
    finally:
        miner_thread.running = False
        miner_thread.join()
        print("Final blockchain status:")
        for block in my_slimcoin.chain:
            print(f"Block #{block.index} Hash: {block.hash}")

if __name__ == "__main__":
    main()

💰 New transaction: {'from': 'Bob', 'to': 'David', 'amount': 14}
💰 New transaction: {'from': 'Alice', 'to': 'Bob', 'amount': 6}
💰 New transaction: {'from': 'Charlie', 'to': 'Alice', 'amount': 6}


Slimcoin network simulation stopped by user. Goodbye!
Final blockchain status:
Block #0 Hash: 7269a20d24a3fa60c942e6bf15a899ffc0754b41a0d120de083eb07a2a3154c0
